based on
https://github.com/midimusicgeneration

In [1]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream

from os import walk
import random
from torch.utils import data
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import random
import matplotlib.pyplot as plt

In [2]:
'''
Music_Data_Utils
functions and classes:
MidiDataset = Torch data loader. Transforms a list of midi files (midi_files) into a pytorch dataloader
list_midi_files = Given a source folder and the number of Train, Validation and test cases that you want, it will return a list of midis for each category
'''

class MidiDataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, qnsf, seq_len=25, cod_type=2, midi_files=[]):
        'Initialization'
        if cod_type !=1 and cod_type !=2:
          raise TypeError("cod_type is not 1 (88 notes) or 2 (176 notes)")
        self.notes = self.get_notes(qnsf=qnsf, cod_type=cod_type, midi_files=midi_files)
        self.qnsf = qnsf
        self.seq_len = seq_len
        self.cod_type = cod_type
        self.midi_source = midi_files



  def __len__(self):
        'Denotes the total number of samples'
        return len(self.notes) - self.seq_len*2

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        samples = self.notes[index:(index+self.seq_len*2)]
        x=np.asarray(samples).astype(np.float32)

        return (x[0:self.seq_len,:],x[self.seq_len:,:])
        #return  (samples[0:self.seq_len], samples[self.seq_len:])

  def get_notes(self, qnsf, cod_type,midi_files):
      """ Get all the notes and chords from the midi files in the ./midi_songs directory """
      notes = []

      for index, file in enumerate(midi_files):
          try:
              print(f"Processing file {file}")  # Print file name
              midi = converter.parse(file)
          except Exception as e:
              print(f"Error processing file {file} (index {index + 1}): {e}")  # Print error message
              continue  # Skip this file and continue with the next one

          try: # file has instrument parts
              s2 = instrument.partitionByInstrument(midi)
              notes_to_parse = s2.parts[0].recurse()
          except: # file has notes in a flat structure
              notes_to_parse = midi.flat.notes

              #initialize piano roll
          length=int(notes_to_parse[-1].offset*qnsf)+1 #count number subdivisions
          notes_song=np.zeros((length,88*cod_type))

          for element in notes_to_parse:
              #print(element.offset, element.duration.quarterLength, element.pitch.midi-21)
              if isinstance(element, note.Note):
                  # cod_type is based on (0,1), (1,0), (0,0)
                  notes_song[int(element.offset*qnsf),cod_type*(element.pitch.midi-21)]=1.0
                  notes_song[(int(element.offset*qnsf)+1):(int(element.offset*qnsf)+int(element.duration.quarterLength*qnsf)),cod_type*(element.pitch.midi-21)+1]=1.0

              elif isinstance(element, chord.Chord):
                  for note_in_chord in element.pitches:
                    # cod_type is based on (0,1), (1,0), (0,0)
                    notes_song[int(element.offset*qnsf),cod_type*(note_in_chord.midi-21)]=1.0
                    notes_song[(int(element.offset*qnsf)+1):(int(element.offset*qnsf)+int(element.duration.quarterLength*qnsf)),cod_type*(note_in_chord.midi-21)+1]=1.0
              #print(notes_song.shape)

          notes+=[list(i) for i in list(notes_song)]
      return notes


def list_midi_files(midi_source, data_len_train, data_len_val , data_len_test, randomSeq=True, seed = 666):
    """ Given a directory with midi files, return a tuple of 3 lists with the filenames for the train, validation and test sets
        The funcion can apply some randomeness in the files selection order although it can be reproduced cause of the seed number
    """
    midi_files_all = []
    # iterate over all filenames in the midi_Source
    for (dirpath, dirnames, filenames) in walk(midi_source):
        midi_files_all.extend(filenames)
    midi_files_all = [ glob.glob(midi_source+"/"+fi)[0] for fi in midi_files_all if fi.endswith(".mid") ]

    # we apply som randomnes in the midi selection to prevent some systematic dependences between train, validation and test.
    if randomSeq:
        random.seed( seed )
        midi_files_sel = random.sample(midi_files_all, (data_len_train + data_len_val + data_len_test))
    else:
        midi_files_sel = midi_files_all
    return midi_files_sel[:data_len_train], midi_files_sel[data_len_train:(data_len_train + data_len_val)], midi_files_sel[(data_len_train + data_len_val):]

def cleanSeq(x, cod_type):
    """ Given a pianoroll x tith 3 dimensions: batches, timesteps and notes*cod_type, the function returns the
        corrected pianoroll adapted to the codificacion rules: a one in the odd position (continuation note),
        has to be precedded in the previous timestep by a one in the even or odd position. If exists, this issue,
        then we moove this one to the even position.
        Another rule is that it can't exist a one in the even and odd position in the same time step. If exists,
        then the one in the ood is removed.
    """
    if cod_type==2:
        # first row correction
        notePos = np.where(x[0,:]==1)[0]        # where are the ones
        notePosWrong = np.array([aa%2 for aa in notePos])
        notePosWrongPos = np.where(notePosWrong==1)[0]     # we look if corespond to odd positions
        x[0,notePos[notePosWrongPos]]=0                    # if present, we correct to 0 the odd
        x[0,notePos[notePosWrongPos]-1]=1                  # if present, we correct to 0 the correspondent even position

        # other rows correction
        # when we have 1 in even and odd position, then remove the second one
        x[:,[ii for ii in range(x.shape[1]) if ii%2==1 ] ] = x[:,[ii for ii in range(x.shape[1]) if ii%2==1 ] ] * (1-x)[:,[ii for ii in range(x.shape[1]) if ii%2==0 ] ]

        # now we want to compare with previous row
        x_0 = x[1:,1:]     #   base values to correct
        x_1 = x[0:-1,1:]   #   previous same column
        x_11 = x[0:-1,0:-1]#   previous even column
        x_sel = np.zeros(x_0.shape)  # odd columns f x
        x_sel[:, [ii for ii in range(x_sel.shape[1]) if ii%2==0 ] ] =1   # corespond to even columns of x_sel
        x_2 = ((x_1==0) & (x_11==0) & (x_0==1) & (x_sel==1))   # columns with one in odd that not have any in even or odd correspondece in previous row
        x[1:,1:][x_2] = 0    # correction in unpair row
        x[1:,:-1][x_2] = 1
    return(x)


def create_midi(prediction_output, qnsf = 4, cod_type=2, midiOutputFile='test_output.mid'):
    """ convert the output (as a numpy array) from the prediction to notes and create a midi file
        from the notes
    """
    offset = 0
    output_notes = []

    prediction_output=cleanSeq(prediction_output, cod_type=2)

    # create note and chord objects based on the values generated by the model
    for i,pattern in enumerate(prediction_output):
        # pattern is a note
        notes = []
        noteIndex = np.where(pattern==1)[0]
        noteIndex = noteIndex[[ii % cod_type ==0 for ii in noteIndex]]
        if len(noteIndex)>0:
            for current_note in noteIndex:
                new_note = note.Note(int(current_note/cod_type + 21))
                new_note.storedInstrument = instrument.Piano()
                new_note.offset = offset
                # duration.quarterLength in case cod_type==2
                if cod_type==2 :
                    # sequence of duration equal to one in the odd position
                    auxDuration = np.where(prediction_output[(i+1):,current_note + 1]==1)[0]
                    # initialize duration
                    minimum_value = 0
                    if len(auxDuration)>0:
                        # minimum position where we have a consecuive sequence at 0
                        # we add one to include complete sequences
                        minimum_value = np.array(range(len(auxDuration)+1))[~np.isin(range(len(auxDuration)+1),auxDuration)].min()
                    # we calcuate the minimum number in the sequance :len(auxDuration) that is not
                    # in the sequence, add one and divide by QNSF
                    new_note.duration.quarterLength = ( minimum_value + 1.0)/qnsf

                output_notes.append(new_note)

        offset += 1.0/qnsf

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=midiOutputFile)
    print("created midi file: ",midiOutputFile )

In [19]:
'''
Seq2Seq
'''

# device definition that depends on the gpu availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Seq2Seq(nn.Module):
	# device definition that depends on the gpu availability
		def __init__(self, input_dim, rnn_dim=512, rnn_layers=2, thr=0):
	# input_dim is the pianoroll size
				super(Seq2Seq, self).__init__()

				self.thr=thr
				self.input_dim=input_dim
				self.rnn_dim=rnn_dim
				self.rnn_layers=rnn_layers

				# encoder decoder definiction based on LSTM with dropout network
				self.encoder = nn.LSTM( input_size=input_dim, hidden_size=rnn_dim, num_layers=rnn_layers, batch_first=True, dropout=0.2)
				self.decoder = nn.LSTM( input_size=input_dim, hidden_size=rnn_dim, num_layers=rnn_layers, batch_first=True, dropout=0.2)
				# the clasification is based on two fully conected networks dropout
				self.classifier = nn.Sequential(
						nn.Linear(rnn_dim, 256),
						nn.ReLU(),
						nn.Dropout(0.5),
						nn.Linear(256, input_dim)
				)

				# loss function
				self.loss_function = nn.BCEWithLogitsLoss() #combines logsoftmax with NLLLoss

		def forward(self, x,y,teacher_forcing_ratio = 0.5):
				# Train method with teacher forcing ratio to introduce more or less real notes in the training step

				# zero padding in the x for the encoder.
				x0 = torch.zeros(x.shape[0], 1, x.shape[2]).to(device)
				x = torch.cat([x0,x], dim=1)

				output, (hn, cn) = self.encoder(x)

				seq_len = y.shape[1]

				# initializing the output to zeros.
				outputs = torch.zeros(y.shape[0], seq_len, self.input_dim).to(device)

				# zero padding in the y for the decoder
				y0 = torch.zeros(y.shape[0], 1, y.shape[2]).to(device)
				y = torch.cat([y0,y], dim=1)

				# initializing the output to zeros.
				input = y[:,0,:].view(y.shape[0],1,y.shape[2])

				# we generate iteratively the future time steps selecting th real o predicted note based on a random scheme
				# conditioned to teacher_forcing_ratio value: 1 is 100% teacher forcing, 0 only predicted notes
				for t in range(1, seq_len):
						output, (hn, cn) = self.decoder(input, (hn, cn))

						teacher_force = random.random() < teacher_forcing_ratio   # teacher forcing draw

						shape = output.shape

						# we add one dimension to classify all bacthes at same time
						x=output.unsqueeze(2)

						x = self.classifier(x) #softmax is'nt necessary

						# removing the extra dimension
						x = x.view(shape[0],shape[1],-1)

						# applying the threshold to the predicted value
						output = (x > self.thr).float()

						# we choose the predicted or real note for the next iteration based on teacher forcing random draw
						input = (y[:,t,:].view(y.shape[0],1,y.shape[2]) if teacher_force else output.view(y.shape[0],1,y.shape[2]))

						# we accumulate the new note
						outputs[:,t,:] = x.view(y.shape[0],-1)

				return outputs

		def loss(self, x,y): #Standard BCE loss
				x = x.view(-1,x.shape[2])
				y_pred = y.view(-1,y.shape[2])

				return self.loss_function(x,y_pred)

		def focal_loss(self, x, y, alpha = 0.5, gamma=2.0): #BCE with focal loss
				x = x.view(-1,x.shape[2])
				y = y.view(-1,y.shape[2])

				t = y.float()

				p = x.sigmoid().detach()
				pt = p*t + (1-p)*(1-t)         # pt = p if t > 0 else 1-p
				w = alpha*t + (1-alpha)*(1-t)  # w = alpha if t > 0 else 1-alpha
				w = w * (1-pt).pow(gamma)
				return F.binary_cross_entropy_with_logits(x, t, w, reduction='sum')

		def recall(self,x,y):
				x_pred = (x > self.thr).long() #if BCELoss expects sigmoid -> th 0.5, BCELossWithLogits expect real values -> th 0.0
				return torch.mul(x_pred.float(),y).float().sum()/y.sum()

		def precision(self,x,y):
				x_pred = (x > self.thr).long() #if BCELoss expects sigmoid -> th 0.5, BCELossWithLogits expect real values -> th 0.0
				return torch.mul(x_pred.float(),y).float().sum()/x_pred.float().sum()

		def training_network(self,training_generator,learning_rate=1e-4,epochs=25, teacher_forcing_val=0.5, tearcher_forcing_strat="fix", focal_alpha=0.75, focal_gamma=2.0):
				#define optimizer
				optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)

				loss_train=[]
				recall_train=[]
				precision_train=[]
				density_train=[]

				for epoch in range(epochs):
						for i,batch in enumerate(training_generator):
								# Forward pass: Compute predicted y by passing x to the model
								x = batch[0]
								y = batch[1]
								x= x.to(device)
								y= y.to(device)

								y_pred = self.train()(x,y,teacher_forcing_ratio=teacher_forcing_val*(1-epoch/epochs)**2)

								# Compute and print loss
								loss = self.focal_loss(y_pred, y, alpha=focal_alpha, gamma=focal_gamma)
								recall  = self.recall(y_pred, y)
								precision = self.precision(y_pred, y)

								# Zero gradients, perform a backward pass, and update the weights.
								optimizer.zero_grad()
								loss.backward()
								optimizer.step()

						loss_train.append(loss.item())

						#how many of the notes on the composition where predicted
						recall_train.append(recall.item())

						#how many of the notes predicted followed the composition
						precision_train.append(precision.item())

						#represents the density of the pianoroll. Good values for 176 (COD_TYPE=2) 4 voices tend to be arround 0.025
						density_train.append((y_pred>self.thr).float().mean().item())

						if epoch%5==0:
								torch.save(self, 'models/model_partial_{0}.pt'.format(epoch))

						print("Epoch: {}, Loss: {:06.2f}, Recall: {:06.4f}, Precision: {:06.4f}, Density: {:06.4f}".format(epoch, loss_train[epoch], recall_train[epoch], precision_train[epoch], density_train[epoch]))

		def zero_init_hidden_predict(self):
				# initialize the hidden state and the cell state to zeros
				# batch size is 1
				return (torch.zeros(self.rnn_layers,1, self.rnn_dim),torch.zeros(self.rnn_layers,1, self.rnn_dim))

		def random_init_hidden_predict(self):
				# initialize the hidden state and the cell state with a normal distribution
				# batch size is 1
				return (torch.randn(self.rnn_layers,1, self.rnn_dim),torch.randn(self.rnn_layers,1, self.rnn_dim))

		def predict(self,seq_len=500,hidden_init="zeros",x_init=torch.zeros(1,1,176).to(device)):

					assert hidden_init=="zeros" or hidden_init=="random" or hidden_init=="guided", "hidden_init can only take values 'zeros', 'random','guided' (in which case you have to provide x_init)"

					self.eval()

					seq = torch.rand(1,seq_len+1,self.input_dim).to(device)

					if hidden_init=="zeros":
							hn,cn=self.zero_init_hidden_predict()
					elif hidden_init=="random":
							hn,cn=self.random_init_hidden_predict()
					else:
							output, (hn,cn)=self.encoder(x_init)
					hn=hn.to(device)
					cn=cn.to(device)

					# for the sequence length
					for t in range(seq_len):

							output, (hn, cn) = self.decoder(seq[0,t].view(1,1,-1),(hn,cn))

							shape = output.shape

							x=output.unsqueeze(2)

							x = self.classifier(x) #no hace falta la softmax

							x = x.view(shape[0],shape[1],-1)

							output = (x > self.thr).float()

							seq[:,t,:] = output.view(shape[0],-1)

					seq = seq[0][1:][:].cpu().detach().numpy()
					return seq


In [4]:
QNSF=4
COD_TYPE=2
SEQ_LEN=128
BATCH_SIZE=16
EPOCHS=20
DATA_LEN_TRAIN=10
DATA_LEN_VAL=5
DATA_LEN_TEST=5
RNN_DIM=512
RNN_LAYERS=2
TEACHER_FORCING=0.5
ALPHA=0.65
GAMMA=2.0
LR=1e-4

In [ ]:
#DataLoader and DataGenerator
MIDI_SOURCE = '/content/drive/MyDrive/midi'
#Read Files (returns a 3-d tuple of list of files)
midi_files = list_midi_files(MIDI_SOURCE, DATA_LEN_TRAIN, DATA_LEN_VAL, DATA_LEN_TEST, randomSeq=True)

#Init Midi Model
dataset_train=MidiDataset(qnsf=QNSF, seq_len=SEQ_LEN, cod_type=COD_TYPE, midi_files=midi_files[0])
dataset_val=MidiDataset(qnsf=QNSF, seq_len=SEQ_LEN, cod_type=COD_TYPE, midi_files=midi_files[1])
dataset_test=MidiDataset(qnsf=QNSF, seq_len=SEQ_LEN, cod_type=COD_TYPE, midi_files=midi_files[2])

Processing file /content/drive/MyDrive/midi/Similar_Trot_00573_Cover_Arrangement_B.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00572_Cover_Timbre_D.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00573_Cover_Genre_A.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00572_Cover_Instrument_D.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00571_Cover_Instrument_D.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00138_Cover_Timbre_A.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00572_Cover_Instrument_B.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00610_Cover_Genre_A.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00572_Cover_Tempo_A.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00571_Cover_Timbre_D.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00571_Cover_Rhythm_C.mid
Processing file /content/drive/MyDrive/midi/Similar_Trot_00585_Cover_Rhythm_C.mid
Pr

In [9]:
BATCH_SIZE_TRAIN=BATCH_SIZE  # batch for the train
BATCH_SIZE_VAL=BATCH_SIZE
BATCH_SIZE_TEST=BATCH_SIZE

#Generate Data Loader
training_generator = data.DataLoader(dataset_train, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
validating_generator = data.DataLoader(dataset_val, batch_size=BATCH_SIZE_VAL, shuffle=True)
testing_generator = data.DataLoader(dataset_test, batch_size=BATCH_SIZE_TEST, shuffle=True)

NameError: name 'dataset_train' is not defined

In [7]:
#Init Model

model = Seq2Seq(input_dim=88*COD_TYPE, rnn_dim=RNN_DIM, rnn_layers=RNN_LAYERS, thr=0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [8]:
#Train Model
import os
# Define the directory path where models will be saved
models_dir = 'models'

# Create the directory if it doesn't exist
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

model.training_network(training_generator,learning_rate=LR,epochs=EPOCHS, teacher_forcing_val=TEACHER_FORCING, tearcher_forcing_strat="fix", focal_alpha=ALPHA, focal_gamma=GAMMA)







NameError: name 'training_generator' is not defined

In [ ]:
torch.save(model, 'models/model_def_{0}_{1}songs.pt'.format(EPOCHS,DATA_LEN_TRAIN))


In [26]:
import os
from torch.utils import data
import torch

def generating(MODEL_PATH,MIDI_FILE,SEQ_LEN=200,HIDDEN_INIT="zeros"):

	device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

	seq2seqMod = torch.load(MODEL_PATH, map_location=device)

	sequence=seq2seqMod.to(device).predict(seq_len=SEQ_LEN,hidden_init=HIDDEN_INIT)

	create_midi(sequence, qnsf = 4, cod_type=2, midiOutputFile=MIDI_FILE)

	return sequence


generating(MODEL_PATH='/content/model_def_20_10songs.pt',SEQ_LEN=500,HIDDEN_INIT='random',MIDI_FILE='/content/output.mid')



created midi file:  /content/output.mid


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.84891284, 0.05509884, 0.12613666, ..., 0.3560839 , 0.14366502,
        0.8119226 ]], dtype=float32)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load('/content/model_def_20_10songs.pt', map_location=device)


# Print the model architecture
print(model)

Seq2Seq(
  (encoder): LSTM(176, 512, num_layers=2, batch_first=True, dropout=0.2)
  (decoder): LSTM(176, 512, num_layers=2, batch_first=True, dropout=0.2)
  (classifier): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=176, bias=True)
  )
  (loss_function): BCEWithLogitsLoss()
)


In [7]:
# Print model parameters
for name, param in model.named_parameters():
    print(f"Parameter: {name}, Shape: {param.shape}")

# Check total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

Parameter: encoder.weight_ih_l0, Shape: torch.Size([2048, 176])
Parameter: encoder.weight_hh_l0, Shape: torch.Size([2048, 512])
Parameter: encoder.bias_ih_l0, Shape: torch.Size([2048])
Parameter: encoder.bias_hh_l0, Shape: torch.Size([2048])
Parameter: encoder.weight_ih_l1, Shape: torch.Size([2048, 512])
Parameter: encoder.weight_hh_l1, Shape: torch.Size([2048, 512])
Parameter: encoder.bias_ih_l1, Shape: torch.Size([2048])
Parameter: encoder.bias_hh_l1, Shape: torch.Size([2048])
Parameter: decoder.weight_ih_l0, Shape: torch.Size([2048, 176])
Parameter: decoder.weight_hh_l0, Shape: torch.Size([2048, 512])
Parameter: decoder.bias_ih_l0, Shape: torch.Size([2048])
Parameter: decoder.bias_hh_l0, Shape: torch.Size([2048])
Parameter: decoder.weight_ih_l1, Shape: torch.Size([2048, 512])
Parameter: decoder.weight_hh_l1, Shape: torch.Size([2048, 512])
Parameter: decoder.bias_ih_l1, Shape: torch.Size([2048])
Parameter: decoder.bias_hh_l1, Shape: torch.Size([2048])
Parameter: classifier.0.weight, 

In [8]:
# Inspect encoder LSTM weights
print(model.encoder.weight_ih_l0)
print(model.encoder.weight_hh_l0)

Parameter containing:
tensor([[ 0.0067,  0.0426,  0.0078,  ...,  0.0282,  0.0234,  0.0057],
        [ 0.0417, -0.0284,  0.0216,  ...,  0.0132,  0.0025,  0.0377],
        [-0.0437, -0.0269, -0.0379,  ..., -0.0279,  0.0163, -0.0096],
        ...,
        [ 0.0149, -0.0027,  0.0034,  ...,  0.0002,  0.0091,  0.0217],
        [-0.0070,  0.0157, -0.0289,  ..., -0.0391,  0.0221, -0.0044],
        [ 0.0214, -0.0301, -0.0361,  ...,  0.0362,  0.0392,  0.0112]],
       requires_grad=True)
Parameter containing:
tensor([[-0.0557, -0.0408, -0.0067,  ..., -0.0248, -0.0149, -0.0476],
        [ 0.0341, -0.0020, -0.0044,  ...,  0.0595, -0.0063,  0.0354],
        [-0.0159, -0.0566,  0.0252,  ...,  0.0271, -0.0408,  0.0065],
        ...,
        [-0.0023, -0.0166,  0.0085,  ..., -0.0004, -0.0369,  0.0081],
        [-0.0229,  0.0308,  0.0076,  ...,  0.0234, -0.0503, -0.0139],
        [-0.0276, -0.0035,  0.0025,  ...,  0.0198, -0.0043,  0.0034]],
       requires_grad=True)


In [1]:
# Assuming predict method is implemented correctly
sequence = model.predict(seq_len=100, hidden_init='random')
print(sequence)

NameError: name 'model' is not defined